In [1]:
import numpy as np
print(np.__version__)
print(np.__file__)

1.24.4
/opt/anaconda3/lib/python3.9/site-packages/numpy/__init__.py


In [12]:
import math
from openmm import unit
from openmmtools import testsystems, states, mcmc
import openmmtools.multistate as multistate
from openmmtools.multistate import ReplicaExchangeSampler
testsystem = testsystems.AlanineDipeptideImplicit()
import os
import tempfile

In [7]:
n_replicas = 3  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]
thermodynamic_states = [states.ThermodynamicState(system=testsystem.system, temperature=T)
                        for T in temperatures]

In [8]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)
simulation = ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=2)


In [16]:
# storage_path = tempfile.NamedTemporaryFile(delete=False).name + '.nc'
storage_path = './alanin_dipeptide.nc'
reporter = multistate.MultiStateReporter(storage_path, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_states,
                  sampler_states=states.SamplerState(testsystem.positions),
                  storage=reporter)

In [17]:
print(storage_path)

./alanin_dipeptide.nc


In [18]:
simulation.run()  # This runs for a maximum of 2 iterations.
simulation.iteration

2

In [19]:
simulation.run(n_iterations=1)
simulation.iteration

2

In [20]:
reporter = multistate.MultiStateReporter(storage=storage_path, open_mode='r', checkpoint_interval=1)
sampler_states = reporter.read_sampler_states(iteration=1)
print(len(sampler_states))

3


In [21]:
print(sampler_states[0].positions.shape)

(22, 3)


In [22]:
import netCDF4

In [ ]:
netCDF4